In [4]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import os as os
os.chdir('/media/shubham/04505A8E505A85F8/Data Science/zs')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
#from sklearn.cross_validation import train_test_split

In [5]:
train = pd.read_csv('train.csv', date_parser=lambda x: pd.to_datetime(x, format="%Y%m"),index_col="Date").reset_index()
test = pd.read_csv('test.csv')

# Data Exploration

In [729]:
len(np.unique(train['PID']))

3000

In [732]:
train['PID'].value_counts().describe()

count    3000.000000
mean      255.595667
std       237.081149
min        11.000000
25%       119.000000
50%       182.000000
75%       312.000000
max      2476.000000
Name: PID, dtype: float64

In [731]:
train.groupby('PID')['Event'].value_counts().describe()

count    285577.000000
mean          2.685045
std           4.434092
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         247.000000
Name: Event, dtype: float64

In [6]:
len(np.unique(train['Event']))

6472

# Starter Code

In [506]:
# starter code
train_dcast = (pd.crosstab(index=[train['PID']], columns=train['Event'])).reset_index()
submit = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit.drop('index',axis=1, inplace=True)
# setting column names in sequence
submit['PID'] = train_dcast['PID']
cols = submit.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit = submit[cols]

In [8]:
train['Date'] = pd.to_datetime(train['Date'])

In [9]:
## extracting year and month from date
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month

# Dividing each year into 8 months and 4 months (Final Model )

In [695]:
#2013-- last 8 months
last8_train = train.loc[(train['year'] == 2013) & (train['month'] > 4)].drop(['year','month'],axis=1)

In [696]:
# selecting patients having more than 10 records
counts = last8_train['PID'].value_counts()
last8_train_records = last8_train[last8_train['PID'].isin(counts[counts >= 10].index)]

In [697]:
# calculating of these much of patients
train_dcast = (pd.crosstab(index=[last8_train_records['PID']], columns=last8_train_records['Event'])).reset_index()
submit1 = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit1.drop('index',axis=1, inplace=True)
# setting column names in sequence
submit1['PID'] = train_dcast['PID']
cols = submit1.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit1 = submit1[cols]

In [698]:
## 2646 patients record
submit1.shape

(2646, 11)

In [699]:
## for rest of the patients which can be available from 2013 starting months
start13_train = train.loc[(train['year'] == 2013) & (train['month'] <= 4)].drop(['year','month'],axis=1)

In [700]:
## patient id present which were not available previous
pids = np.setdiff1d(start13_train['PID'],last8_train_records['PID'])
pids = pd.DataFrame(data = pids, columns = ['PID'])

In [702]:
# data of 2013
t13 = train.loc[train['year'] == 2013]

In [703]:
# getting info of those ids
left_last8_train = (pd.merge(pids,t13,on='PID', how='left'))

In [704]:
# removing patient data which have less than 10 records
counts = left_last8_train['PID'].value_counts()
left_last8_train_records = left_last8_train[left_last8_train['PID'].isin(counts[counts >= 10].index)]

In [705]:
# calculating of these much of patients
train_dcast = (pd.crosstab(index=[left_last8_train_records['PID']], columns=left_last8_train_records['Event'])).reset_index()
submit2 = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit2.drop('index',axis=1, inplace=True)
# setting column names in sequence
submit2['PID'] = train_dcast['PID']
cols = submit2.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit2 = submit2[cols]

In [706]:
## 203 patient data
submit2.shape

(203, 11)

In [ ]:
## similarly getting data from the previous year

In [707]:
last12_train = train.loc[(train['year'] == 2012) & (train['month'] > 4)].drop(['year','month'],axis=1)

In [708]:
final1 = submit1.append(submit2)

In [709]:
left_train12 = pd.DataFrame(data = np.setdiff1d(last12_train['PID'],final1['PID']), columns= ['PID'])

In [710]:
t12_13 = train.loc[(train['year'] == 2013) | ((train['year'] == 2012) & (train['month'] > 4))]

In [711]:
left_train12 = (pd.merge(left_train12,t12_13,on='PID',how='left'))

In [712]:
## removing events occuring less than 10
counts = left_train12['PID'].value_counts()
left_train12_records = left_train12[left_train12['PID'].isin(counts[counts >= 10].index)]

In [714]:
# calculating of these much of patients
train_dcast = (pd.crosstab(index=[left_train12_records['PID']], columns=left_train12_records['Event'])).reset_index()
submit3 = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit3.drop('index',axis=1, inplace=True)
# setting column names in sequence
submit3['PID'] = train_dcast['PID']
cols = submit3.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit3 = submit3[cols]

In [715]:
submit3.shape

(103, 11)

In [716]:
final2 = final1.append(submit3)

In [717]:
left_pid1 = pd.DataFrame(np.setdiff1d(submit['PID'],final2['PID']) ,columns= ['PID'])

In [718]:
train_after11 = train.loc[(train['year'] == 2012) | (train['year'] == 2013)]
left_pid_data = pd.merge(left_pid1, train_after11,on = 'PID', how='left')

In [719]:
left_pid_records = left_pid_data.dropna()

In [720]:
#
counts = left_pid_records['PID'].value_counts()
left_pid_data_records = left_pid_records[left_pid_records['PID'].isin(counts[counts >= 10].index)]

In [721]:
# calculating of these much of patients
train_dcast = (pd.crosstab(index=[left_pid_data_records['PID']], columns=left_pid_data_records['Event'])).reset_index()
submit4 = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit4.drop('index',axis=1, inplace=True)
# setting column names in sequence
submit4['PID'] = train_dcast['PID']
cols = submit4.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit4 = submit4[cols]

In [ ]:
## merging all the dataframes 

In [722]:
final3 = final2.append(submit4)

In [723]:
last_ones = pd.DataFrame(np.setdiff1d(submit['PID'],final3['PID']), columns= ['PID'])

In [724]:
# getting rest prediction from 2011 data
final = final3.append(pd.merge(last_ones,submit,on='PID',how='left'))

In [725]:
final.shape

(3000, 11)

In [726]:
final.to_csv('final.csv', index=False)

# segementation of patients based on frequency

In [7]:
## freq visitors and less freq wise
counts = train['PID'].value_counts()
freqt = train[train['PID'].isin(counts[counts > 181].index)]

In [8]:
frqt = freqt[freqt['year'] == 2013].drop('year',axis=1)

In [9]:
counts = train['PID'].value_counts()
less_freqt = train[train['PID'].isin(counts[counts < 182].index)]

In [10]:
## preparing cross tab for both and then append
train_dcast = (pd.crosstab(index=[freqt['PID']], columns=freqt['Event'])).reset_index()
fqt = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
fqt.drop('index',axis=1, inplace=True)
# setting column names in sequence
fqt['PID'] = train_dcast['PID']
cols = fqt.cdolumns.tolist()
cols = cols[-1:] + cols[:-1]
fqt = fqt[cols]

In [11]:
train_dcast = (pd.crosstab(index=[less_freqt['PID']], columns=less_freqt['Event'])).reset_index()
less_fqt = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
less_fqt.drop('index',axis=1, inplace=True)
# setting column names in sequence
less_fqt['PID'] = train_dcast['PID']
cols = less_fqt.columns.tolist()
cols = cols[-1:] + cols[:-1]
less_fqt = less_fqt[cols]

In [12]:
frq_wise = fqt.append(less_fqt)

In [16]:
frq_wise = frq_wise.sort_values('PID')

In [17]:
frq_wise.to_csv('freq_wise.csv',index=False)

In [18]:
frq_wise.shape

(3000, 11)

In [8]:
len(np.unique(train.loc[train['year'] == 2013,'PID']))

2965

# Giving respective weights to year wise event that occur

In [8]:
train_11 = train.loc[train['year'] == 2011].drop('year',axis=1)
train_dcast11 = (pd.crosstab(index=[train_11['PID']], columns=train_11['Event'])).reset_index()

In [9]:
train_dcast11.shape

(2868, 5118)

In [10]:
train_dcast.shape

(3000, 6473)

In [11]:
col = np.setdiff1d(train_dcast.columns,train_dcast11.columns)

In [12]:
t11 = pd.concat([train_dcast11, pd.DataFrame(columns = col)]).fillna(0)

In [13]:
pid = np.setdiff1d(train_dcast['PID'],train_dcast11['PID'])

In [14]:
t_11= t11.append(pd.DataFrame(data=pid, columns= ['PID'])).sort_values('PID').reset_index().drop('index',axis=1).fillna(0)*0.1

In [15]:
train_12 = train.loc[train['year'] == 2012].drop('year',axis=1)
train_dcast12 = (pd.crosstab(index=[train_12['PID']], columns=train_12['Event'])).reset_index()
col = np.setdiff1d(train_dcast.columns,train_dcast12.columns)
t12 = pd.concat([train_dcast12, pd.DataFrame(columns = col)]).fillna(0)
pid = np.setdiff1d(train_dcast['PID'],train_dcast12['PID'])
t_12 = t12.append(pd.DataFrame(data=pid, columns= ['PID'])).sort_values('PID').reset_index().drop('index',axis=1).fillna(0)*0.2

In [16]:
train_13 = train.loc[train['year'] == 2013].drop('year',axis=1)
train_dcast13 = (pd.crosstab(index=[train_13['PID']], columns=train_13['Event'])).reset_index()
col = np.setdiff1d(train_dcast.columns,train_dcast13.columns)
t13 = pd.concat([train_dcast13, pd.DataFrame(columns = col)]).fillna(0)
pid = np.setdiff1d(train_dcast['PID'],train_dcast13['PID'])
t_13 = t13.append(pd.DataFrame(data=pid, columns= ['PID'])).sort_values('PID').reset_index().drop('index',axis=1).fillna(0)*0.7

In [17]:
df1 = pd.DataFrame.add(t_11,t_12,axis=1)

In [18]:
df2 = pd.DataFrame.add(df1,t_13,axis=1)

In [19]:
pid_order = test.sort_values('PID').reset_index().drop('index',axis=1)

In [20]:
fqt = df2.loc[:,df2.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
fqt.drop('index',axis=1, inplace=True)
# setting column names in sequence
fqt['PID'] = pid_order['PID']
cols = fqt.columns.tolist()
cols = cols[-1:] + cols[:-1]
fqt = fqt[cols]

In [21]:
fqt.to_csv('weights.csv',index=False)